<a href="https://colab.research.google.com/github/acts701/Audio/blob/main/241118_speechToText_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install google-cloud-speech
from google.cloud import speech_v2
from google.oauth2 import service_account
import os
import sqlite3
import datetime
import openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.0/305.0 kB 4.2 MB/s eta 0:00:00


In [3]:
from google.colab import userdata

# Set up Google Cloud credentials
credentials = userdata.get('googleApiKey')

# Google 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Initialize the Speech-to-Text client
client = speech_v2.SpeechClient(credentials=credentials)

In [10]:
from google.cloud.speech_v2 import SpeechClient
from google.cloud.speech_v2.types import cloud_speech
PROJECT_ID = os.getenv("spatial-garage-424313-s5")

# def quickstart_v2(audio_file: str) -> cloud_speech.RecognizeResponse:
def transcribe_audio(input_file, output_file):
    """Transcribe an audio file.
    Args:
        audio_file (str): Path to the local audio file to be transcribed.
    Returns:
        cloud_speech.RecognizeResponse: The response from the recognize request, containing
        the transcription results
    """
    # Reads a file as bytes
    with open(input_file, "rb") as f:
        audio_content = f.read()

    # Instantiates a client
    client = SpeechClient()

    config = cloud_speech.RecognitionConfig(
        auto_decoding_config=cloud_speech.AutoDetectDecodingConfig(),
        language_codes=["ko-KR"],
        model="long",
    )

    request = cloud_speech.RecognizeRequest(
        recognizer=f"projects/{PROJECT_ID}/locations/global/recognizers/_",
        config=config,
        content=audio_content,
    )

    # Transcribes the audio into text
    response = client.recognize(request=request)

    for result in response.results:
        print(f"Transcript: {result.alternatives[0].transcript}")

    with open(output_file, 'w', encoding='utf-8') as text_file:
        for result in response.results:
            text_file.write(result.alternatives[0].transcript + '\n')

    return response

In [16]:
# def transcribe_audio(input_file, output_file):
#     """
#     Transcribe an audio file using Google Cloud Speech-to-Text v2 API
#     and save the transcription to a text file.

#     :param input_file: Path to the input audio file (mp3 format)
#     :param output_file: Path to save the transcribed text
#     """
#     # Load the audio file
#     with open(input_file, 'rb') as audio_file:
#         audio_content = audio_file.read()

#     # Configure the request
#     config = speech_v2.RecognitionConfig(
#         language_codes=["ko-KR"],  # Korean language
#         audio_encoding="MP3",  # Match your file encoding
#         sample_rate_hertz=44100,  # Set your audio sample rate
#     )

#     # Perform the transcription
#     request = {
#         "config": config,
#         "audio": {"content": audio_content},
#     }

#     response = client.recognize(request=request)

#     # Extract and save the transcription
#     with open(output_file, 'w', encoding='utf-8') as text_file:
#         for result in response.results:
#             text_file.write(result.alternatives[0].transcript + '\n')

In [6]:
def process_and_store_transcription(output_file, db_path):
    """
    Process the transcription text, tokenize, translate, and store in SQLite database.

    :param output_file: Path to the transcription text file
    :param db_path: Path to the SQLite database file
    """
    # Read the transcription text
    with open(output_file, 'r', encoding='utf-8') as file:
        sentences = file.readlines()

    # Connect to the database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Create table if it doesn't exist
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS translations (
        id INTEGER,
        file_name TEXT,
        insert_date DATE,
        original_text TEXT,
        engine TEXT,
        PRIMARY KEY (id, file_name, insert_date, original_text, engine)
    )
    ''')

    # Process each sentence
    for sentence in sentences:
        original_text = sentence.strip()
        if not original_text:
            continue

        # Translate to English using OpenAI API
        translated_text = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Translate the following Korean text to English."},
                {"role": "user", "content": original_text}
            ]
        )["choices"][0]["message"]["content"]

        # Tokenize the text
        tokenized_text = " ".join(original_text.split())

        # Generate embeddings (mocked for now)
        embedding = "embedding_placeholder"

        # Insert into database
        cursor.execute('''
        INSERT INTO translations (id, file_name, insert_date, original_text, engine)
        VALUES (?, ?, ?, ?, ?)
        ''', (
            None,  # id will be auto-incremented if set to NULL
            os.path.basename(output_file),
            datetime.datetime.now().strftime('%Y-%m-%d'),
            original_text,
            "Google Speech-to-Text v2"
        ))

    # Commit and close the connection
    conn.commit()
    conn.close()

In [11]:
# Define input, output, and database file paths
input_file = os.path.expanduser("/content/drive/My Drive/Isaiah/20160608(수요_사1장10-20절).mp3")
output_file = os.path.expanduser("/content/drive/My Drive/llm/data/voiceToText/googlePaid/20160608(수요_사1장10-20절).txt")
db_path = os.path.expanduser("/content/drive/My Drive/llm/data/db/translationsV2.db")

# Transcribe, process, and store the output
transcribe_audio(input_file, output_file)
process_and_store_transcription(output_file, db_path)

print(f"Transcription completed and stored in database: {db_path}")

ERROR:grpc._plugin_wrapping:AuthMetadataPluginCallback "<google.auth.transport.grpc.AuthMetadataPlugin object at 0x7fb146753760>" raised exception!
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/credentials.py", line 128, in refresh
    self._retrieve_info(request)
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/credentials.py", line 101, in _retrieve_info
    info = _metadata.get_service_account_info(
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/_metadata.py", line 323, in get_service_account_info
    return get(request, path, params={"recursive": "true"})
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/_metadata.py", line 248, in get
    raise exceptions.TransportError(
google.auth.exceptions.TransportError: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Go

ServiceUnavailable: 503 Getting metadata from plugin failed with error: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Engine metadata service. Status: 404 Response:\nb''", <google.auth.transport.requests._Response object at 0x7fb146302950>)